In [1]:
import urllib, sys
from bs4 import BeautifulSoup
import re
from collections import Counter
import time
import csv
import pandas as pd
from urllib.error import HTTPError


In [2]:
def scrape_data_df(symbol_list,wordlist):
    
#''GE','PEP', 'MHCP',STT
    #symbol_list=['GOOG', 'GE','ETN','PEP','KHC','FB','MCHP','SLAB','JPM','STT']

    #creating a list to store the data for different symbols
    temp_list = []

    for symbol in symbol_list:

        site='http://seekingalpha.com/symbol/'+symbol+'/earnings/transcripts'
            #     print(symbol)
        hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                   'Accept-Encoding': 'none',
                   'Accept-Language': 'en-US,en;q=0.8',
                   'Connection': 'keep-alive'} 
        pageReq=urllib.request.Request(site,headers=hdr) 
        page=urllib.request.urlopen(pageReq).read()
        soup=BeautifulSoup(page,'lxml')

        for link in soup.find_all('a'):

            x=link.get('href')
            if isinstance(x,str):

                        # Parameterization Pending
                        #wordlist=['transcript','2018','q3']

                if all(x.find(s)>=0 for s in wordlist):
                    parse_site='http://seekingalpha.com/'+x
                    parse_site='http://seekingalpha.com/'+x+'?part=single'
                    print('Extracting Call Transcripts from: ' + symbol)
                    print(parse_site)

                            # Open the URL as Browser, not as python urllib
                    pageAct=urllib.request.Request(parse_site,headers=hdr) 
                    infileAct=urllib.request.urlopen(pageAct).read()

                    soup = BeautifulSoup(infileAct, 'html5lib') 

                            #creating an empty list
                    text = []

                    try:
                        #saving the scraped data in a list for all the symbols    
                        text.append(soup.find_all('div', attrs = {'id':'a-body'}))

                    except HTTPError as e:
                            print('Error Code', e.code)

                           #looping through data for different symbols in the 'text' list
                    for elements in text:

                                #creating an empty dictionary
                        df_dict = {}

                                #appending the data of different symbols to dictionary
                        df_dict['Ticker'] = symbol
                        df_dict['Year'] = wordlist[1]
                        df_dict['Quarter'] = wordlist[2]
                        df_dict['PublishedTime'] = "".join([time.text for time in soup.find_all("time", itemprop="datePublished")])
                        df_dict['summary'] = "".join([texts.text for texts in soup.find_all("p")])
                        df_dict['title'] = "".join([title.text for title in soup.find_all("title")])

                                #converting the dictionary to list 
                        temp_list.append(df_dict)

            #saving the list as a dataframe               
    df2= pd.DataFrame(temp_list)

            #coverting the dataframe output to a csv file
    df2.to_csv("output.csv")

            #increase the time depending on the symbols we are downloading the data for
    time.sleep(120)

    return df2

In [3]:
# ,'ETN','PEP','KHC','FB','MCHP','SLAB','JPM','STT'
scrape_data_df(['GOOG', 'GE','ETN','PEP','KHC','FB','MCHP','SLAB','JPM','STT'],['transcript','2018','q3'])

Extracting Call Transcripts from: GOOG
http://seekingalpha.com//article/4214694-alphabet-goog-q3-2018-results-earnings-call-transcript?part=single


HTTPError: HTTP Error 403: Forbidden